In [1]:
import matplotlib.pyplot as plt
import folium
import pandas as pd
import numpy as np

In [2]:
def read_data(file_name):
    with open(file_name) as csv:
        lines = csv.readlines()
        for i in range(len(lines)):
            lines[i] = lines[i].split(',')
        columns = lines[0]
        data = lines[1:]
    return columns, data

In [3]:
def extract_pos(columns, data):
    lats = [float(line[1]) for line in data]
    longs = [float(line[2]) for line in data]
    positions = list(zip(lats, longs))
    return positions

In [4]:
columns, data = read_data('resources/dados_BTSs.csv')
bts_positions = extract_pos(columns, data)
center_lat = np.mean([x for x, _ in bts_positions])
center_long = np.mean([x for _, x in bts_positions])
print(center_lat, center_long)

-8.07118981488889 -34.89566203706295


In [141]:
center_lat = np.mean([lat for lat, _ in bts_positions])
center_long = np.mean([long for _, long in bts_positions])
folium_map = folium.Map(location=[center_lat, center_long],
                        zoom_start=13)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(folium_map)

In [5]:
columns, data = read_data('resources/LocTreino_Equipe_2.csv')
positions = extract_pos(columns, data)

In [143]:
for location in positions:
    marker = folium.CircleMarker(location=location)
    marker.add_to(folium_map)

In [144]:
folium_map.save("resources/my_map.html")

In [193]:
folium_map

In [6]:
from sklearn import linear_model

In [20]:
def extract_features(data):
    features = []
    targets = []
    for sample in data:
        try:
            features.append([float(x) for x in sample[3:]])
            targets.append([float(x) for x in sample[1:3]])
        except:
            pass
    return features, targets

In [21]:
samples, targets = extract_features(data)

In [22]:
lat_reg = linear_model.LinearRegression(normalize=True)
lat_reg.fit(samples, [x for x, _ in targets])

long_reg = linear_model.LinearRegression(normalize=True)
long_reg.fit(samples, [x for _, x in targets])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [23]:
predicted_lat = lat_reg.predict(samples)
predicted_long = long_reg.predict(samples)
# print(predicted_lat)

In [24]:
# for p_lat, p_long, (lat, long) in list(zip(predicted_lat, predicted_long, targets)):
#     print(100*(p_lat - lat)/lat, 100*(p_long - long)/long)
# plt.plot(predicted_lat, [x for x, _ in targets], marker='x', linestyle='None')
# plt.show()
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

def rmse(y_pred, y_real):
    return sqrt(mse(y_pred, y_real))

print(rmse(predicted_lat, [x for x, _ in targets]))
print(rmse(predicted_long, [x for _, x in targets]))

0.001319815519781799
0.001663067757033124


In [83]:
# from folium.plugins import MarkerCluster
folium_map = folium.Map(location=[center_lat, center_long],
                        zoom_start=13,
                        tiles='CartoDB dark_matter')
# cluster = MarkerCluster(list(zip(predicted_lat, predicted_long))).add_to(folium_map)
n_points = 10000
predicted_positions = list(zip(predicted_lat, predicted_long))
for location in predicted_positions[:n_points]:
    marker = folium.CircleMarker(location=location, color='red', weight=1, radius=3)
    marker.add_to(folium_map)
for location in positions[:n_points]:
    marker = folium.CircleMarker(location=location, color='blue', weight=1, radius=3)
    marker.add_to(folium_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(folium_map)
for p1, p2 in list(zip(positions, predicted_positions))[:n_points]:
    folium.PolyLine([p1, p2], color="green", weight=1, opacity=0.5).add_to(folium_map)
print(len(positions), len(predicted_lat))
folium_map.save("resources/my_map.html")

1500 1497
